# 供给信号理解Pipeline - 多行业自动识别版本
本Notebook支持自动识别数据中的category列，为每个行业动态生成对应的prompt并进行推理

In [2]:
import pandas as pd
from tqdm import tqdm
import json
from utils import friday_api, own_model_api
from datetime import datetime
import os
import re

# 全局配置参数

In [3]:
# 数据集路径
DATA_PATH = "../data/供给信号文本挖掘评测集V2_for_review_updated.xlsx"

# Friday API 配置
formatted_date = datetime.now().strftime("%Y%m%d")
tenantId = "433017463651568"
model_id = "12969"
model_version = '2'
misId = 'zhangyuntao06'
model_name = 'Qwen3-30B-A3B-Instruct-2507-Meituan'  # 使用新的Qwen3模型
gpu_team = 'hmart-generalshop'
gpu_queue = 'root.zw05_training_cluster.hadoop-generalshop.gpu'
gpu_type = 'A100-80G'
gpu_num = 2
batch_size = 4096
max_new_tokens = 2048

# 行业特定prompt参数

In [4]:
# 行业服务范畴定义
HANGYE_FANCHOU = {
    "美甲": "涉及服务包含1）美甲服务：本甲上色、指甲延长，款式设计等；2）基础护理：甲面抛光，指甲修型等。注意美睫相关服务不属于美甲行业。",
    "健身中心": "涉及服务包含1）提供健身场所；2）课程/培训；3）健身效果。注意，健身运动后的相关物品，衣着等不属于行业范畴。",
    "台球": "涉及服务包含1）场馆服务：提供场地设备；2）专业服务：助教/陪练服务等。",
    "保姆": "涉及服务包含1）家庭家务服务：家电清洗，衣物洗涤等；2）母婴护理服务：产妇护理，新生儿护理等；3）养老护理服务：健康护理，精神陪伴等；4）病患陪护服务：医院陪护等。不包含针对物品的介绍。",
    "运动培训": "仅指各项运动培训服务中的课程内容、教学服务和教学场地。知识科普等不在范畴内。",
    "洗浴": "涉及服务包含1）桑拿洗浴服务：泡汤，按摩；2）提供场地设施内容：汗蒸房、娱乐休闲区。"
}

# 行业CPV属性字典
CPV_DICT = {
    "健身中心": """**F属性分类判定规则**（重要）：
1. **健身器材**：指具体的设备/器械名称本身
   - 包括：跑步机、史密斯架、蝴蝶机、哑铃、杠铃、划船机、椭圆机、龙门架、卧推架、腿举机、筋膜枪等
   - 不包括：使用器械的动作名称
2. **课程类别**：指训练方式、训练动作、课程类型的完整名称
   - 包括：力量训练、有氧训练、瑜伽、普拉提、HIIT、卧推、深蹲、硬拉、肩部训练、背部训练、帕梅拉、健身操、古法健身操等
   - 提取规则：保持完整的训练方式/动作名称，不要拆解。包含"训练"、"操"、"课"等的具体名称都应提取。
3. **锻炼部位**：仅指身体部位名称
   - 包括：全身、臀部、核心、背部、腿部、肩部、胸部、手臂、腰部、腹部等
4. **授课形式**：一对一私教、一对二私教、团课、线上直播课、训练营等
5. **场馆设施**：泳池、有氧区、力量区、淋浴间、私教房等
6. **场馆环境**：每日消毒、空气净化、室内隔音好、超大面积场馆、独立私教房、落地窗、智能门禁等
7. **场馆类型**：无人自助健身房、学生健身房、女性专属健身房、普拉提馆、瑜伽馆、私教工作室等
8. **门店/器材品牌**：威尔士、一兆韦德、乐刻运动、超级猩猩、KEEP、中田健身、Technogym、匹克等
9. **营业时间**：全天可用、早间特惠时段、深夜时段等
10. **是否需要预约**：需要预约、不需要预约、仅团课需预约等
11. **收费模式**：次卡、月卡、季卡、年卡、按次付费等
12. **服务内容**：体质测试、InBody测试、饮食指导、体脂率检测等
13. **附加权益**：赠送体验课、饮食指导、亲友体验券、免费提供洗浴用品等
14. **教练资质**：ACE认证、NSCA认证、ACSM认证等
15. **教练经验**：10年以上教学经验、资深教学经验等
16. **教练性别**：美女教练、男教练等""",
    
    "台球": """**F属性分类判定规则**（重要）：
1. **培训课程**：零基础入门、进阶技术提升、斯诺克专项、考级培训等
2. **赛事支持**：相关赛事信息
3. **设备器材**：进口球桌、高端球杆、美式台球桌、私人球杆柜、自动摆球机、计分显示屏等
4. **环境设施**：独立包厢、禁烟区、有酒吧、VIP包房等
5. **环境风格**：美式复古、现代简约、赛博朋克风、网红风等
6. **场馆环境**：室内隔音好、室内宽敞明亮、超大面积场馆、全面禁烟等
7. **场馆类型**：无人自助球馆、无烟球馆、智能门店等
8. **球杆配置**：品牌公杆、碳素公杆、定期维护/更换皮头等
9. **球桌配置**：赛台(金腿/银腿)、钢库桌、TV台(比赛专用)、加热台(斯诺克必备)等
10. **球杆/球桌品牌**：Predator(美洲豹)、Mezz(美兹)、Fury(威利)、野豹、环球、乔氏球桌、星牌球桌、Brunswick(宾士域)、Diamond(钻石)等
11. **营业时间**：24小时营业、夜间等
12. **是否需要预约**：需要预约、不需要预约等
13. **收费模式**：小时计费、按局计费、畅打套餐等
14. **服务内容**：按小时计费、助教陪练等
15. **教练资质**：台协认证教练、国家级教练、省级运动员等
16. **助教陪练**：美女陪练、专业陪练、初级助教、资深助教、明星助教、职业选手陪练等""",
    
    "美甲": """**F属性分类判定规则**（重要）：
1. **款式风格**：纯色、跳色、法式(传统/变体)、猫眼(晶石/宽猫)、渐变/晕染、腮红甲、新中式、Y2K/辣妹风、老钱风/极简、ins风、手绘款等
2. **甲型**：方形、方圆形、杏仁甲、梯形甲/芭蕾甲、尖甲等
3. **品牌**：Presto、VETRO、Ageha、国产胶等
4. **颜色色系**：红色、裸色系(透色/妈生甲)、美拉德色系(大地色)、多巴胺色系(高饱和)、莫兰迪色系、显白深色系、金属色系、极光粉、魔镜粉等
5. **甲油胶质地**：实色胶、透色胶、冰透/果冻、爆闪/亮片、铂金等
6. **延长方式**：贴片延长(全贴/半贴)、纸托光疗延长、水晶延长、琉璃延长等
7. **装饰元素/饰品**：施华洛世奇钻、锆石、珍珠、金属链条、蝴蝶结等
8. **图案设计**：棋盘格、波点、3D堆胶、水波纹、浮雕等
9. **封层/光泽度**：超亮封层、磨砂封层等
10. **热门IP**：Hello Kitty、美少女战士、疯狂动物城、鬼灭之刃等
11. **门店环境**：ins网红装修、私密性强等
12. **设备设施**：美甲躺椅(电动沙发)、独立包间、影视区、宠物友好、新风系统等
13. **技师等级**：初级美甲师、资深美甲师、店长/技术总监等
14. **服务项目**：本甲护理、本甲上色等
15. **服务部位**：手部、足部等
16. **附加服务**：修眉、手部去角质、手部SPA、足部去死皮/脚后跟护理、嵌甲矫正等
17. **适合场景**：通勤、婚礼、旅行、海边、度假、新年、圣诞等""",
    
    "运动培训": """**F属性分类判定规则**（重要）：
1. **场地设施**：国际标准比赛场地、专业运动木地板、塑胶/地胶场地、室外草坪、室内恒温泳池、露天场地等
2. **场馆环境**：独立淋浴间、干湿分离更衣室、休息区等
3. **课程类型**：启蒙兴趣班、基础技能班、体验课/试听课、考证班、寒暑假训练营等
4. **教学内容**：篮球、足球、羽毛球、游泳、滑雪等
5. **授课形式**：私教、小班、大班等
6. **教练资质**：国家二级运动员、退役职业运动员、国际认证(如ACE/NSCA/ITF等)、救生员证/急救证等
7. **培训目标**：兴趣培养、体能增强、技能掌握、减肥塑形、升学体育考试、职业考证、运动康复等
8. **教学风格**：实战导向、趣味互动型、启发式教学等
9. **培训课时**：10节、20节等""",
    
    "洗浴": """**F属性分类判定规则**（重要）：
1. **功能区域**：公共休息大厅、胶囊房、榻榻米、观影区、游戏区、全天候自助餐、水果畅吃等
2. **装修风格**：现代简约、日式原木等
3. **汤泉种类**：碳酸汤、绢丝浴、牛奶浴、室外露天汤泉、日式缸浴、儿童戏水池、按摩池、冷水池、盐浴、药浴、桑拿房、芬兰干蒸等
4. **洗护用品**：国际大牌洗护、全套一次性用品、提供高档护肤品/爽肤水、提供化妆品、戴森吹风机等
5. **服务项目**：搓背、足疗/足道、中式推拿、精油SPA、采耳、修脚、拔罐/刮痧、醋搓、过夜服务等
6. **适用场景**：情侣约会、家庭聚会、团建聚会等""",
    
    "保姆": """**F属性分类判定规则**（重要）：
1. **人员类型**：住家保姆、月嫂、育儿嫂等
2. **人员年龄**：25-35岁、35-45岁、45-50岁、50岁以上等
3. **资质证书**：母婴护理师证、育婴师证、保育员证、健康证、驾驶证、营养师证等
4. **擅长菜系**：家常菜、川湘菜、粤菜、面食、西餐简餐、月子餐、宝宝辅食等
5. **核心技能**：早教互动、产后康复指导、催乳按摩、小儿推拿、收纳整理、宠物照料等
6. **工作内容**：新生儿护理、产妇护理、保洁等
7. **从业经验**：1-3年、3-5年、10年以上等
8. **学历水平**：小学、初中、高中/中专、大专及以上、本科等
9. **服务模式**：24小时住家、白班(早出晚归)、住家、月嫂、做六休一、仅工作日等
10. **性格特点**：活泼开朗、温和有耐心、话少干练、喜欢宠物、不怕猫狗等
11. **健康状况**：幽门螺杆菌阴性、乙肝五项正常、每年常规体检等
12. **服务对象**：新生儿、早产儿、产妇等"""
}

# FAB属性分类指引（通用）
FAB_GUIDANCE = """**F属性（客观特征）**：
- 定义：严格按照上述F属性分类判定规则进行挖掘
- 核心判断：客观属性（无主观评价），属性值为文本原文

**A属性（主观优点）**：
- 定义：对客观事物的主观感受和评价性描述
- 判断标准：形容词或评价性短语，无程度副词（很/超级/巨/特别等）
- 例子："专业"、"性价比高"、"环境好"、"服务热情"

**B属性（场景利益）**：
- 定义：描述能够达成的效果、解决的问题、适用的场景或人群
- 具体分类：
  * 人群/身份标签：指适用的具体人群或身份
  * 场景：指适用的具体情境或场合
  * 目的/效果：指能达成的具体效果或解决的问题
- 例子:"适合初学者"、"缓解压力"、"亲子互动"、"商务宴请"等"""

# Prompt生成函数

In [5]:
def generate_prompt(content_text, hangye):
    """为指定行业生成FAB挖掘的prompt"""
    
    # 获取行业相关配置
    hangye_scope = HANGYE_FANCHOU.get(hangye, "")
    cpv_attributes = CPV_DICT.get(hangye, "")
    
    prompt = f"""# 你的角色
你是本地生活行业的专家，有着丰富的供给理解知识。你需要基于FAB模型理论，从文本内容中挖掘{hangye}行业下的FAB属性。

# FAB模型介绍
F（特征/Feature）：客观属性。是产品具备的客观物理属性，是人们对商品进行辨识的信息因素，比如产品的品牌、材料、工艺、尺寸、颜色等。
A（优点/Advantage）：主观属性。将客观物理属性提炼为产品优点或者作用，不同用户对于该属性的解读拥有千人千面的主观解读。
B（利益/Benefit）：主观属性。阐释产品能够满足的具体场景下的需求，可以理解为某种用处，适用于某种场景/人群。

# 核心任务
从文本中提取符合{hangye}行业标准的FAB属性，并输出为JSON格式。行业范畴：{hangye_scope}

# 行业客观属性及分类规则
{cpv_attributes}

# FAB属性分类指引
{FAB_GUIDANCE}

# 关键判别规则（必须遵守）
1. **有效性过滤**：
   - 舍弃宽泛无意义词汇（如：有效、好、不错、舒服、棒）
   - 舍弃程度副词（如：超级、特别、很、巨、极其、十分、比较、更、最）
   - 舍弃负向/吐槽内容
2. **结构与内容规范**：
   - **A/B属性（主观）**：需提炼为规范短语（动宾结构或名词短语），要求连续、非口语化。
   - **F属性（客观）**：必须截取原文，且需包含明确主体。
3. **格式约束**：
   - 禁止包含任何标点符号。
   - 单个属性值长度必须小于8个字。

# 输出格式
你的输出格式为JSON格式，严格按照示例格式输出，其中reasoning字段表示你的推理过程，推理过程需要严格按照任务顺序进行推理并得出最终结果；result为FAB属性提取的结果，result字段是一个list，list中的每个元素是一个dict，一个dict对应不同的FAB属性，type字段表示属性类型，attribute表示对应的值。
**重要**：当分类为F时，你需要提取相应的属性类型和属性值，对于同一属性类型的多个属性值，请使用列表格式，如{{"课程类别": ["瑜伽", "普拉提"]}}。当分类为A或B时，如提取到多个属性词，则用','分割，如果没有对应的分类的属性，则attribute字段值为"无"。再次提醒，F属性值为文本原文，A/B属性为提炼后的短语。
示例输出：
{{"reasoning": "推理过程", "result": [{{"type": "F", "attribute": {{"属性类别": ["属性值1", "属性值2"]}}}}, {{"type": "A", "attribute": "文本1,文本2"}}, {{"type": "B", "attribute": "无"}}]}}

# 待处理文本
文本内容：{content_text}
输出：
"""
    return prompt

# 加载数据并识别行业

In [8]:
# 加载数据
print(f"正在加载数据：{DATA_PATH}")
data = pd.read_excel(DATA_PATH).fillna('')
print(f"✓ 数据加载成功，共 {len(data)} 条记录")
print(f"✓ 列名：{list(data.columns)}")

# 检测category列
if 'category' not in data.columns:
    print("✗ 错误：数据中不存在 'category' 列")
    raise ValueError("数据中必须包含 'category' 列")

# 识别所有行业
industries = data['category'].unique()
print(f"\n✓ 检测到 {len(industries)} 个行业：{list(industries)}")

# 检查是否所有行业都有配置
unconfigured_industries = [ind for ind in industries if ind not in HANGYE_FANCHOU]
if unconfigured_industries:
    print(f"⚠ 警告：以下行业未在配置中：{unconfigured_industries}")
    print("这些行业将被跳过")

正在加载数据：../data/供给信号文本挖掘评测集V2_for_review_updated.xlsx
✓ 数据加载成功，共 200 条记录
✓ 列名：['category', 'content', 'F', 'A', 'B']

✓ 检测到 6 个行业：['保姆', '美甲', '台球', '洗浴', '运动培训', '健身中心']


# 生成统一的推理数据

In [9]:
# 生成所有行业的推理数据到一个JSON文件
print(f"\n{'='*70}")
print("开始生成所有行业的推理数据...")
print(f"{'='*70}\n")

all_friday_samples = []
industry_stats = {}

for hangye in industries:
    # 跳过未配置的行业
    if hangye not in HANGYE_FANCHOU:
        print(f"⚠ 跳过未配置的行业：{hangye}")
        continue
    
    # 过滤该行业的数据
    industry_data = data[data['category'] == hangye].reset_index(drop=True)
    data_count = len(industry_data)
    industry_stats[hangye] = data_count
    
    print(f"正在处理 {hangye}：{data_count} 条数据")
    
    # 为该行业的每条数据生成prompt
    for i in tqdm(range(len(industry_data)), desc=f"  生成{hangye}数据"):
        content = industry_data.iloc[i, 1]  # content列是第1列
        prompt = generate_prompt(content, hangye)
        
        friday_sample_line_dict = {}
        friday_sample_line_dict['input'] = [
            {"role": "system", "content": f"你是一个专业的本地生活行业FAB属性挖掘专家，专注于{hangye}行业。请严格遵守行业特定规则，并在reasoning中体现判断过程。"},
            {'role': 'user', 'content': prompt}
        ]
        friday_sample_line_dict['target'] = ''
        all_friday_samples.append(friday_sample_line_dict)

# 保存统一的JSON文件
json_file = f"all_industries_xhs_zhukeguan_wajue_{formatted_date}.json"
print(f"\n正在保存统一的JSON文件...")
with open(json_file, mode='w', encoding='utf-8') as f:
    for sample in all_friday_samples:
        # ensure_ascii=False可以保证写到文件里的中文不会变成类似这样的编码\u4ef7\u503c98
        json.dump(sample, f, ensure_ascii=False)
        f.write('\n')

print(f"✓ JSON文件已保存：{json_file}")
print(f"✓ 总共生成 {len(all_friday_samples)} 条推理数据")
print(f"\n各行业数据统计：")
for hangye, count in industry_stats.items():
    print(f"  - {hangye}: {count} 条")


开始生成所有行业的推理数据...

正在处理 保姆：30 条数据


  生成保姆数据: 100%|██████████| 30/30 [00:00<00:00, 28155.99it/s]


正在处理 美甲：30 条数据


  生成美甲数据: 100%|██████████| 30/30 [00:00<00:00, 22180.35it/s]


正在处理 台球：30 条数据


...

正在处理 运动培训：30 条数据


  生成运动培训数据: 100%|██████████| 30/30 [00:00<00:00, 30541.05it/s]


正在处理 健身中心：50 条数据


  生成健身中心数据: 100%|██████████| 50/50 [00:00<00:00, 31799.12it/s]


正在保存统一的JSON文件...
✓ JSON文件已保存：all_industries_xhs_zhukeguan_wajue_20260106.json
✓ 总共生成 200 条推理数据

各行业数据统计：
  - 保姆: 30 条
  - 美甲: 30 条
  - 台球: 30 条
  - 洗浴: 30 条
  - 运动培训: 30 条
  - 健身中心: 50 条


In [18]:
# 上传到HDFS
print(f"\n{'='*70}")
print("正在上传到HDFS...")
!hdfs dfs -put -f $json_file viewfs://hadoop-meituan/user/hadoop-hmart-generalshop/yaoyu13/xhs/
print(f"✓ 文件已上传到HDFS")


正在上传到HDFS...
✓ 文件已上传到HDFS


In [19]:
# 调用Friday API（只调用一次）
print(f"\n{'='*70}")
print("正在调用Friday API进行统一推理...")
task_name = f"xhs_all_industries_prompt_task_sft_V2"
response = own_model_api(
    task_name=task_name,
    tenantId=tenantId,
    json_file=f'viewfs://hadoop-meituan/user/hadoop-hmart-generalshop/yaoyu13/xhs/{json_file}',
    misId=misId,
    model_id=model_id,
    model_version=model_version,
    gpu_team=gpu_team,
    gpu_queue=gpu_queue,
    gpu_type=gpu_type,
    gpu_num=gpu_num,
    batch_size=batch_size,
    max_new_tokens=max_new_tokens
)

print(f"\n{'='*70}")
print("✓✓✓ 推理任务已提交！✓✓✓")
print(f"{'='*70}")
print(f"\n任务名称: {task_name}")
print(f"数据文件: {json_file}")
print(f"总数据量: {len(all_friday_samples)} 条")
print(f"\n请到 Friday 平台查看任务状态，完成后记录任务ID用于下载结果。")


正在调用Friday API进行统一推理...
{"code":0,"data":321404,"message":"成功"}

✓✓✓ 推理任务已提交！✓✓✓

任务名称: xhs_all_industries_prompt_task_sft_V2
数据文件: all_industries_xhs_zhukeguan_wajue_20260106.json
总数据量: 200 条

请到 Friday 平台查看任务状态，完成后记录任务ID用于下载结果。


# 下载并解析结果
⚠️ 注意：需要等待Friday API推理完成后再执行此部分

你可以在Friday平台查看任务状态，完成后记录任务ID

In [24]:
# 跟据结果地址下载json文件
!hdfs dfs -get viewfs://hadoop-meituan/user/hadoop-hmart-generalshop/friday/eval_out/321622_xhs_all_industries_prompt_task_sft_V3.json ./infer_results/

# 解析结果

In [25]:
# 文本挖掘
import re
import json
import pandas as pd
from tqdm import tqdm

# 读取原始标注数据
origin_data = pd.read_excel("../data/供给信号文本挖掘评测集V2_for_review_updated.xlsx")

# 读取模型推理结果
file = open("/home/sankuai/dolphinfs_zhangyuntao06/daily_January/1.6/code/infer_results/321622_xhs_all_industries_prompt_task_sft_V3.json", 'r', encoding='utf-8')

deal_result = []
for line in tqdm(file.readlines()):
    dic = json.loads(line)
    deal_result.append(dic)

pred_list = []
reasoning_list = []
count = 0
for i in range(len(deal_result)):
    response = deal_result[i]['output']['content']
    if "```json" in response:
        json_str = response.split("```json")[1].split("```")[0].strip()
    elif "{" in response and "}" in response:
        start = response.find("{")
        end = response.rfind("}") + 1
        json_str = response[start:end]
    else:
        json_str = response
    try:
        # 提取F属性
        f_attr_str = ""
        try:
            f_match = re.search(r'"type"\s*:\s*"F".*?"attribute"\s*:\s*({[^}]+})', json_str, re.DOTALL)
            if f_match:
                f_attr_dict_str = eval(f_match.group(1))
                for p,v in f_attr_dict_str.items():
                    v_list = str(v).replace("[","").replace("]","").split(",")
                    for v in v_list:
                        f_attr_str += p + ":" + v + ","
        except:
            count += 1
            f_attr_str = "无"
    
        # 提取A和B属性
        a_result = re.findall(r'"type":"A","attribute":"(.*?)"', json_str.replace(" ","").replace("\n",""))
        b_result = re.findall(r'"type":"B","attribute":"(.*?)"', json_str.replace(" ","").replace("\n",""))
        a_attr = a_result[-1] if len(a_result) > 0 else "无"
        b_attr = b_result[-1] if len(b_result) > 0 else "无"

        # 提取reasoning字段（模型推理过程）
        try:
            reasoning_match = re.search(r'"reasoning"\s*:\s*"(.*?)"(?:,|$)', json_str, re.DOTALL)
            if reasoning_match:
                reasoning_text = reasoning_match.group(1)
                # 处理转义字符
                reasoning_text = reasoning_text.replace('\\n', '\n').replace('\\"', '"')
                reasoning_list.append(reasoning_text)
            else:
                reasoning_list.append("")
        except:
            reasoning_list.append("")

        pred_list.append([f_attr_str.rstrip(","), a_attr, b_attr])
    except KeyboardInterrupt:
        raise
    except:
        count += 1
        pred_list.append(["无", "无", "无"])
        reasoning_list.append("")
            
# 添加预测结果列
origin_data['pred_F'] = [item[0] for item in pred_list]
origin_data['pred_A'] = [item[1] for item in pred_list]
origin_data['pred_B'] = [item[2] for item in pred_list]
origin_data['reasoning'] = reasoning_list
 
# 保存为xlsx文件
output_file = "./results/3_hangye_sft_V3.xlsx"
origin_data.to_excel(output_file, index=False)

print(f"解析内容失败数量：{count}/{len(deal_result)}")
print(f"结果已保存到: {output_file}")
print(f"\n前5行预览:")
print(origin_data[['category', 'content', 'F', 'A', 'B', 'pred_F', 'pred_A', 'pred_B', 'reasoning']].head())


100%|██████████| 200/200 [00:00<00:00, 19781.19it/s]

解析内容失败数量：0/200
结果已保存到: ./results/3_hangye_sft_V3.xlsx

前5行预览:
  category                                            content  F  A  B  \
0       保姆  52岁阿姨的陆冲向前Pumping。第一次户外上课，一镜到底的向前Pumping。教练说现在...  无  无  无   
1       保姆  阿姨自己买的不给吃？。#家政改变女人的生活 #月嫂育婴师 #迷茫与成长 #月嫂育婴师养老护理...  无  无  无   
2       保姆  搬家不头疼！大家电搬运保姆级攻略。很多小伙伴在问：“大家电搬家怎么搬才不会翻车？”别慌！今天...  无  无  无   
3       保姆  保洁阿姨可是精得很呀！早知道收拾好再退房。#令人想笑的offer #搞笑 #反转 #vlog...  无  无  无   
4       保姆  避雷烟台月嫂YBY! !。烟台芝罘区月嫂姚××在我家干了八天半我忍无可忍让她走了，后悔没早点...  无  无  无   

                                              pred_F          pred_A  \
0                                                                  无   
1                  人员类型:'月嫂',人员类型: '育婴师',服务内容:'养老护理'               无   
2  服务内容:'大家电搬运',服务内容: '精细打包',服务内容: '还原整理',服务模式:'全...  服务透明,精细打包,专业团队   
3                                                                  无   
4  服务模式:'24小时住家',服务模式: '白班',人员类型:'月嫂',工作内容:'新生儿护理...               无   

              pred_B                                        

# 过滤模块

In [26]:
filter_input_file = "./results/3_hangye_sft_V3.xlsx"
filter_output_file = "./results/3_hangye_sft_V3_filtered.xlsx"

def parse_f_attribute_string(f_str):
    """
    解析 F 属性（字符串格式 - 用于人工标注数据）

    Args:
        f_str: F 属性字符串

    Returns:
        属性值列表
    """
    try:
        if not f_str or f_str == '无' or f_str == '':
            return []
        f_str = str(f_str).strip()
        items = [item.strip() for item in f_str.replace('，', ',').split(',')]
        result = []
        for item in items:
            if item and item != '无':
                if '：' in item:
                    result.append(item.split('：')[1].strip())
                elif ':' in item:
                    result.append(item.split(':')[1].strip())
                else:
                    result.append(item)
        return result
    except Exception as e:
        return []


def parse_ab_attribute(ab_str):
    """
    解析 A/B 属性（逗号分隔格式）

    Args:
        ab_str: A/B 属性字符串

    Returns:
        属性值列表
    """
    try:
        if not ab_str or ab_str == '无' or ab_str == '':
            return []
        ab_str = str(ab_str).replace('，', ',')
        return [item.strip() for item in ab_str.split(',') if item.strip()]
    except Exception as e:
        return []


def parse_pred_f_attribute(f_str):
    """
    解析预测的 F 属性（冒号分隔格式）

    Args:
        f_str: F 属性字符串（格式：key:value,key:value,...）

    Returns:
        属性值列表
    """
    try:
        if not f_str or f_str == '无' or f_str == '':
            return []
        f_str = str(f_str).strip()
        items = [item.strip() for item in f_str.split(',')]
        result = []
        for item in items:
            if item and item != '无':
                if ':' in item:
                    result.append(item.split(':')[1].strip())
                else:
                    result.append(item)
        return result
    except Exception as e:
        return []


print("\n" + "="*80)
print("过滤模块")
print("="*80)

# 初始化过滤规则
filters = []

# 1. 过滤地名（如果文件存在）
try:
    address_file = "../data/地点全集.xlsx"
    if os.path.exists(address_file):
        address = pd.read_excel(address_file, sheet_name="Sheet1")
        address_list = address['address'].tolist()
        if address_list:
            address_filter = '|'.join(address_list)
            filters.append(address_filter)
            print(f"✓ 地名过滤器已加载，共 {len(address_list)} 个地名")
except:
    print("⚠ 地名过滤器加载失败或文件不存在")

# 2. 过滤比较副词
adv = ['很', '非常', '极了', '较', '极其', '有点', '极端', '极度', '特别', '十分',
       '格外', '尤其', '极为', '超级', '异常', '无比', '绝对', '完全', '稍微', '稍稍',
       '稍许', '多少', '略微', '还算', '微微', '一点', '一些', '几乎', '差不多', '简直',
       '近乎', '差点', '差一点', '彻底', '毫不', '丝毫', '根本', '全然']
adv_filter = '|'.join(adv)
filters.append(adv_filter)
print(f"✓ 副词过滤器已加载，共 {len(adv)} 个副词")

# 3. 过滤通用泛词
common = [
    "运动健身", "好的教练", "有效", "好", "不错", "舒服", "棒",
    "很好", "特别好", "超级好", "挺好", "还不错", "还可以",
    "效果好", "效果不错", "效果很好", "不错的", "很不错"
]
common_filter = '|'.join(common)
filters.append(common_filter)
print(f"✓ 泛词过滤器已加载，共 {len(common)} 个泛词")

# 合并所有过滤器
all_filter = '|'.join(filters)

# 读取解析模块的输出文件
print("\n读取解析结果文件...")

filter_data = pd.read_excel(filter_input_file)
print(f"✓ 已读取 {len(filter_data)} 条记录")

# 统计过滤结果
from collections import Counter

removed_f_words = []  # 被移除/过滤掉的词
removed_a_words = []
removed_b_words = []

for idx in range(len(filter_data)):
    label_f_list = parse_f_attribute_string(filter_data.iloc[idx]['F'])
    label_a_list = parse_ab_attribute(filter_data.iloc[idx]['A'])
    label_b_list = parse_ab_attribute(filter_data.iloc[idx]['B'])

    # 收集所有被移除的词（匹配过滤规则的词）
    for word in label_f_list:
        if re.search(all_filter, word):
            removed_f_words.append(word)

    for word in label_a_list:
        if re.search(all_filter, word):
            removed_a_words.append(word)

    for word in label_b_list:
        if re.search(all_filter, word):
            removed_b_words.append(word)

# 统计被移除的词汇频次
f_counter = Counter(removed_f_words)
a_counter = Counter(removed_a_words)
b_counter = Counter(removed_b_words)

# 获取过滤词数
f_filtered_count = len(removed_f_words)
a_filtered_count = len(removed_a_words)
b_filtered_count = len(removed_b_words)

# 生成过滤后的文件（保留基础列）

filter_data.to_excel(filter_output_file, index=False)

# 输出统计信息
print(f"\n✓ 过滤完成，结果已保存到: {filter_output_file}\n")

# F属性统计 - 显示被过滤掉的最多的3个词
print(f"【F属性 (特征)】")
print(f"  过滤词数: {f_filtered_count}")
top_3_f = f_counter.most_common(3)
print(f"  过滤最多的3个词: {', '.join([f'{word}' for word, count in top_3_f])}")

# A属性统计 - 显示被过滤掉的最多的3个词
print(f"\n【A属性 (优点)】")
print(f"  过滤词数: {a_filtered_count}")
top_3_a = a_counter.most_common(3)
print(f"  过滤最多的3个词: {', '.join([f'{word}' for word, count in top_3_a])}")

# B属性统计 - 显示被过滤掉的最多的3个词
print(f"\n【B属性 (利益)】")
print(f"  过滤词数: {b_filtered_count}")
top_3_b = b_counter.most_common(3)
print(f"  过滤最多的3个词: {', '.join([f'{word}' for word, count in top_3_b])}")



过滤模块
✓ 地名过滤器已加载，共 34907 个地名
✓ 副词过滤器已加载，共 38 个副词
✓ 泛词过滤器已加载，共 18 个泛词

读取解析结果文件...
✓ 已读取 200 条记录

✓ 过滤完成，结果已保存到: ./results/3_hangye_sft_V3_filtered.xlsx

【F属性 (特征)】
  过滤词数: 11
  过滤最多的3个词: 日本胶, 马卡龙美甲, 新加坡桑拿

【A属性 (优点)】
  过滤词数: 6
  过滤最多的3个词: 宝宝护理很专业, 手法非常娴熟, 服务很顶

【B属性 (利益)】
  过滤词数: 6
  过滤最多的3个词: 放松解压好去处, 下班放松好去处, 新加坡周末好去处输出
